# Faster data manipulation and vectorizing with pandas and NumPy

This iPython notebooks is inspired by PyCon talk: Original video [here](https://youtu.be/nxWginnBklU).

In [1]:
import pandas as pd
import numpy as np
import re
import time

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('max_columns', 15)
pd.set_option('chained_assignment', None)

## Read in and setup the data 
This CSV is an example of the data I worked on while practicing learning how to vectorize former `.apply` function. 
The data here have been scrubbed and scrambled to not relate to any real-life data.

In [2]:
df = pd.read_csv('sample_data_pygotham2019.csv', 
                 parse_dates=['Date Created', 
                              'Original Record: Date Created'
                             ]
                )

### Data inspection 

In [3]:
df.shape
df.head(5)

(273399, 14)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,Original Record: Date Created,Date Created,Inactive,Specialty,Providers,duplicate_leads,bad_leads
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,False,True
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,True,True
2,16,6936,Practice,Nonpaid : Webinar : AdvancedFax and Patient : ...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-12-03 17:33:00,No,Dermatology,0.0,False,True
3,16,4557,Practice,Nonpaid : Webinar : AdvancedFax and Patient : ...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-12-03 17:33:00,No,Dermatology,0.0,True,True
4,20,8637,Practice,Nonpaid : Webinar : HIPAA Audit Prep : LSID 53...,- None -,CLIENT-Live,CLIENT-Live,2003-04-30 18:15:00,2016-08-24 14:09:00,No,Plastic Surgery,1.0,False,False


We visualize how many different types of columns are in the dataframe

In [4]:
df.dtypes

Internal ID                               int64
ID                                        int64
Category                                 object
Lead Source                              object
Lead Source Category                     object
Current Status                           object
Status at Time of Lead                   object
Original Record: Date Created    datetime64[ns]
Date Created                     datetime64[ns]
Inactive                                 object
Specialty                                object
Providers                               float64
duplicate_leads                            bool
bad_leads                                  bool
dtype: object

In [5]:
list(set(df['Current Status']))

['LEAD-General Pool',
 'LEAD-Nurture',
 'PROSPECT-LONG TERM',
 'CLIENT-Reseller',
 'PROSPECT-NO DECISION',
 'CLIENT-Referring Consultant',
 'LEAD-Claim',
 'LEAD-Contact Program',
 'CLIENT-Live',
 'LEAD-Long-Term Opportunity',
 'LEAD-3 Flame No Contact',
 'PROSPECT-CURRENT',
 'LEAD-No Contact',
 'CLIENT-Implementation (EMR)',
 'CLIENT-Closed-Sold',
 'PROSPECT-LOST',
 'CLIENT-Transferred',
 'LEAD-Campaign',
 'CLIENT-Deactivated',
 'LEAD-Unqualified',
 'CLIENT-Handoff',
 'CLIENT-Partner',
 'LEAD-Test Lead',
 'LEAD-Vendor - Bad Lead',
 'CLIENT-Implementation',
 'LEAD-Engaged',
 'LEAD-Bad Info']

# First attempt at vectorizing with conditionals
Suppose we have to convert the - None - value in the column 'Current Status' as follows:

In [6]:
def set_lead_status(row):
    if row['Current Status'] == '- None -':
        return row['Status at Time of Lead']
    else:
        return row['Current Status']

In [7]:
%%timeit
test = df.apply(set_lead_status, axis=1)

1 loop, best of 3: 22.3 s per loop


This is relatively slow. We can vectorize this by passing if/else statements to columns

In [8]:
def set_lead_status(col1, col2):
    if col1 == '- None -':
        return col2
    else:
        return col1

In [9]:
%%timeit
try: 
    test1 = set_lead_status(df['Current Status'], df['Status at Time of Lead'])
except: 
    pass

10 loops, best of 3: 36.5 ms per loop


Notice that .apply can make your whole code seriously slow. 
Any time you see .apply in a code, that should be your first opportunity for a speed gain. 
There is almost always a method available to eliminate .apply in a python code. 

# Elegant way: Enter `numpy.where()`
[Documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html)
Similar to Excel 'IF' function, you give it a condition, except this condition can handle the entire array/column in a vectorized manner. Then give it what to return if elements on that boolean array are true or false.

In [10]:
%%timeit
# you can pass the output directly into a pandas Series

df['normalized_status'] = np.where(
                                    df['Status at Time of Lead'] == '- None -',   # <-- condition
                                    df['Current Status'],                         # <-- return if true
                                    df['Status at Time of Lead']                  # <-- return if false
                                   )

10 loops, best of 3: 50.5 ms per loop


We can make this even faster by accessing the underlying NumPy arrays from pandas series.
This makes it faster b/c now there's only a NumPy array of your data to pass to C, with no need for handling all the stuff attached to a pandas Series that makes them so convenient to work with. The method to achieve this is to use .values as shown below

In [11]:
%%timeit

df['normalized_status'] = np.where(
    df['Status at Time of Lead'].values == '- None -',
    df['Current Status'].values, 
    df['Status at Time of Lead'].values
)

100 loops, best of 3: 20.5 ms per loop


Notice the speed gains in the above. Our .apply method took 10 seconds. the np.where with .values took 10 micro seconds. 

# What about `numpy.vectorize()`?

This is a function that will take a Python function and turn it into a NumPy ufunc, so it can handle vectorized approaches. It _vectorizes_ your function, not necessarily how that function applies to your data. There is a difference here. Sometimes a function could be an elaborate converter, such as latitude/longitude to MGRS converter is a multiple hundred line floating point calculation and converting to np.where is not applicable in a straightforward manner. To solve such problems, significant speed gains can be achieved using numpy.vectorize()

Resources:
 - https://docs.scipy.org/doc/numpy/reference/generated/numpy.vectorize.html
 - https://www.experfy.com/blog/why-you-should-forget-loops-and-embrace-vectorization-for-data-science


In [12]:
# Here is our previous function that we tried to vectorize but couldn't due to the ValueError. 
def works_fast_maybe(col1, col2):
    if col1 == '- None -':
        return col2
    else:
        return col1

In [13]:
# with the np.vectorize method --> returns a vectorized callable
vectfunc = np.vectorize(works_fast_maybe)

In [14]:
%%timeit 
test3 = list(vectfunc(df['Status at Time of Lead'], df['Current Status']))

10 loops, best of 3: 187 ms per loop


Notice, that the speed went from 10 seconds to roughly 80 micro seconds, but not exactly 10 micro seconds. Still it is much slower than np.where

In [15]:
def can_I_go_any_faster(status_at_time, current_status):
    # this works fine if we are setting static values
    df['test'] = 'test'# status_at_time
    df.loc[status_at_time == '- None -', 'test'] = current_status  # <-- ValueError, indexes don't match!
    df.loc[status_at_time != '- None -', 'test'] = status_at_time

In [16]:
%%timeit
test4 = can_I_go_any_faster(df['Status at Time of Lead'], df['Current Status'])

1 loop, best of 3: 136 ms per loop


In [17]:
def can_I_go_any_faster2(status_at_time, current_status):
    # this works fine if you're setting static values
    df['test'] = 'test'# status_at_time
    df.loc[status_at_time == '- None -', 'test'] = 'statys1_isNone' 
    df.loc[status_at_time != '- None -', 'test'] = 'statys2_notNone'

In [18]:
%%timeit
test5 = can_I_go_any_faster2(df['Status at Time of Lead'], df['Current Status'])

10 loops, best of 3: 125 ms per loop


# Multiple conditions

### lead_category

In [19]:
list1 = ['LEAD-3 Flame No Contact', 'LEAD-Campaign', 'LEAD-Claim', 'LEAD-Contact Program', 
         'LEAD-General Pool', 'LEAD-No Contact', 'LEAD-Nurture', 'LEAD-Unqualified', 'PROSPECT-LOST']

list2 = ['- None -', 'CLIENT-Closed-Sold', 'CLIENT-Handoff', 'CLIENT-Implementation', 'CLIENT-Implementation (EMR)',
         'CLIENT-Live', 'CLIENT-Partner', 'CLIENT-Referring Consultant', 'CLIENT-Transferred', 'LEAD-Engaged', 
         'LEAD-Long-Term Opportunity', 'PROSPECT-CURRENT', 'PROSPECT-LONG TERM', 'PROSPECT-NO DECISION']

# apply version
def lead_category(row):
    if row['Original Record: Date Created'] == row['Date Created']:
        return 'New Lead'
    elif row['normalized_status'].startswith('CLI'):
        return 'Client Lead'
    elif row['normalized_status'] in list1:
        return 'MTouch Lead'
    elif row['normalized_status'] in list2:
        return 'EMTouch Lead'
    return 'NA'

In [20]:
%%timeit
df['lead_category0'] = df.apply(lead_category, axis=1)

1 loop, best of 3: 17.4 s per loop


You can call a `np.where` for every condition, and it will run fine. But it gets a little hard to read after a while.

In [21]:
%%timeit
df['lead_category'] = np.where(df['Original Record: Date Created'].values == df['Date Created'].values, 'New Lead', 
            np.where(df['normalized_status'].str.startswith('CLI').values, 'Client Lead', 
                    np.where(df['normalized_status'].isin(list1).values, 'MTouch Lead', 
                            np.where(df['normalized_status'].isin(list2).values, 'EMTouch Lead', 
                                     'NA') 
                                  )
                         )
                )

10 loops, best of 3: 151 ms per loop


# Enter `numpy.select()`
Notice that the np.where code is not very tractable. It gets nested although it is fast. 
An elegant way to run multiple conditions is to use numpy.select()

[Documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html)

In [22]:
%%timeit
conditions = [
    df['Original Record: Date Created'].values == df['Date Created'].values,
    df['normalized_status'].str.startswith('CLI').values,
    df['normalized_status'].isin(list1).values,
    df['normalized_status'].isin(list2).values
]

choices = [
    'New Lead', 
    'Client Lead', 
    'MTouch Lead',
    'EMTouch Lead'
]


df['lead_category1'] = np.select(conditions, choices, default='NA')  # Order of operations matter!

10 loops, best of 3: 158 ms per loop


In [23]:
# Their output logic is the same
(df.lead_category == df.lead_category1).all()

True

## What about nested multiple conditionals? Can we vectorize that?
Yes, this can be done easily using numpy.select()

In [24]:
# This is something you might think you can't vectorize, but you sure can!
def sub_conditional(row):
    if row['Inactive'] == 'No':
        if row['Providers'] == 0:
            return 'active_no_providers'
        elif row['Providers'] < 5:
            return 'active_small'
        else:
            return 'active_normal'
    elif row['duplicate_leads']:
        return 'is_dup'
    else:
        if row['bad_leads']:
            return 'active_bad'
        else:
            return 'active_good'

In [25]:
%%timeit
# Let's time how long it takes to apply a nested multiple condition func
df['lead_type'] = df.apply(sub_conditional, axis=1)

1 loop, best of 3: 14.5 s per loop


In [26]:
%%timeit

# With np.select, could do .values here for additional speed, but left out to avoid too much text
conditions = [
    ((df['Inactive'] == 'No') & (df['Providers'] == 0)),
    ((df['Inactive'] == 'No') & (df['Providers'] < 5)),
    df['Inactive'] == 'No',
    df['duplicate_leads'],  # <-- you can also just evaluate boolean arrays
    df['bad_leads'],
]

choices = [
    'active_no_providers',
    'active_small',
    'active_normal',
    'is_dup',
    'active_bad',
]

df['lead_type_vec'] = np.select(conditions, choices, default='NA')

10 loops, best of 3: 84.4 ms per loop


In [28]:
mask = (
    ((df['Inactive'] == 'No') & (df['Providers'] == 0))
    & ((df['Inactive'] == 'No') & (df['Providers'] < 5))
    & (df['Inactive'] == 'No')
    & (df['duplicate_leads'])  # <-- you can also just evaluate boolean arrays
    & df['bad_leads']
)


In [29]:
%%timeit

# With np.select
conditions = [
    ((df['Inactive'].values == 'No') & (df['Providers'].values == 0)),
    ((df['Inactive'].values == 'No') & (df['Providers'].values < 5)),
    df['Inactive'].values == 'No',
    df['duplicate_leads'].values,  # <-- you can also just evaluate boolean arrays
    df['bad_leads'].values,
]

choices = [
    'active_no_providers',
    'active_small',
    'active_normal',
    'is_dup',
    'active_bad',
]

df['lead_type_vec'] = np.select(conditions, choices, default='NA')

C:\Users\malshe\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in less
  """


10 loops, best of 3: 50.7 ms per loop


# What about more complicated things?
Of course this is just a tiny sample of things you might encounter, but I thought they could be useful to see how vectorization can still apply even with otherwise difficult cases.

#### Let us look at string manipulations

In [30]:
df.head(2)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,...,normalized_status,test,lead_category0,lead_category,lead_category1,lead_type,lead_type_vec
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,CLIENT-Deactivated,statys1_isNone,Client Lead,Client Lead,Client Lead,active_no_providers,active_no_providers
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,CLIENT-Deactivated,statys1_isNone,Client Lead,Client Lead,Client Lead,active_no_providers,active_no_providers


In [31]:
# Doing a regex search to find string patterns

def find_paid_nonpaid(s):
    if re.search(r'non.*?paid', s, re.I):
        return 'non-paid'
    elif re.search(r'Buyerzone|^paid\s+', s, re.I):
        return 'paid'
    else:
        return 'unknown'

In [32]:
%%timeit
# our old friend .apply()
df['lead_source_paid_unpaid'] = df['Lead Source'].apply(find_paid_nonpaid)

1 loop, best of 3: 1.15 s per loop


How does `np.vectorize()` do for strings?

In [33]:
%%timeit

vect_str = np.vectorize(find_paid_nonpaid)

df['lead_source_paid_unpaid1'] = vect_str(df['Lead Source'])

1 loop, best of 3: 914 ms per loop


In [38]:
%%timeit
# How does a list comprehension do?
df['lead_source_paid_unpaid2'] = ['non-paid' if re.search(r'non.*?paid', s, re.I) 
                                  else 'paid' if re.search(r'Buyerzone|^paid\s+', s, re.I) 
                                  else 'unknown' for s in df['Lead Source']]

1 loop, best of 3: 794 ms per loop


pandas provides the `.str()` methods for working with strings.

In [39]:
%%timeit
# How does this compare?
conditions = [
    df['Lead Source'].str.contains(r'non.*?paid', case=False, na=False),
    df['Lead Source'].str.contains(r'Buyerzone|^paid\s+', case=False, na=False),
]

choices = [
    'non-paid',
    'paid'
]

df['lead_source_paid_unpaid1'] = np.select(conditions, choices, default='unknown')

1 loop, best of 3: 770 ms per loop


*what about not searching strings?*

In [40]:
%%timeit
df['lowerls'] = df['Lead Source'].apply(lambda x: x.lower())

10 loops, best of 3: 139 ms per loop


In [41]:
%%timeit
df['lowerls1'] = df['Lead Source'].str.lower()

10 loops, best of 3: 149 ms per loop


#### Dictionary lookups

In [34]:
channel_dict = {
    'Billing Service': 'BS', 
    'Consultant': 'PD', 
    'Educational': 'PD', 
    'Enterprise': 'PD', 
    'Hospital': 'PD', 
    'IPA': 'PD', 
    'MBS': 'RCM', 
    'MSO': 'PD', 
    'Medical practice': 'PD', 
    'Other': 'PD', 
    'Partner': 'PD',
    'PhyBillers': 'BS', 
    'Practice': 'PD', 
    'Purchasing Group': 'PD',
    'Reseller': 'BS', 
    'Vendor': 'PD', 
    '_Other': 'PD', 
    'RCM': 'RCM'
}

def a_dict_lookup(row):
    if row['Providers'] > 7:
        return 'Upmarket'
    else:
        channel = channel_dict.get(row['Category'])
        return channel

In [35]:
%%timeit
df['dict_lookup'] = df.apply(a_dict_lookup, axis=1)

1 loop, best of 3: 8.34 s per loop


In [38]:
%%timeit
df['dict_lookup1'] = np.where(
    df['Providers'].values > 7, 
    'Upmarket',
    df['Category'].map(channel_dict)
)

C:\Users\malshe\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:258: RuntimeWarning: invalid value encountered in greater


10 loops, best of 3: 32.7 ms per loop


In [39]:
%%timeit
channel_values = df['Category'].map(channel_dict)
df['dict_lookup1'] = np.where(
    df['Providers'] > 7, 
    'Upmarket',
    channel_values
)

10 loops, best of 3: 32.3 ms per loop


In [40]:
%%timeit
# Using np.vectorize to vectorize a dictionary .get() method works, but is slower than .map()
channel_values = np.vectorize(channel_dict.get)(df['Category'].values)
df['dict_lookup2'] = np.where(
    df['Providers'] > 7, 
    'Upmarket',
    channel_values
)

10 loops, best of 3: 42.8 ms per loop


#### Date operations

In [42]:
df.head(2)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,...,lead_type,lead_type_vec,lead_source_paid_unpaid,lead_source_paid_unpaid1,dict_lookup,dict_lookup1,dict_lookup2
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,active_no_providers,active_no_providers,non-paid,non-paid,PD,PD,PD
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,...,active_no_providers,active_no_providers,non-paid,non-paid,PD,PD,PD


In [43]:
# make a new column called 'Start Date' for dummy testing
# ONly do a fraction so we have some NaN values
df['Start Date'] = df['Date Created'].sample(frac=0.8)

In [44]:
def weeks_to_complete(row):
    """Calculate the number of weeks between two dates"""
    if pd.isnull(row['Start Date']):
        return (row['Original Record: Date Created'] -  row['Date Created']).days / 7
    else:
        return (row['Date Created'] - row['Start Date']).days / 7

In [45]:
%%timeit
wtc1 = df.apply(weeks_to_complete, axis=1)

1 loop, best of 3: 25.1 s per loop


One approach to vectorization is to use pandas `.dt` datetime accessors. 

In [47]:
%%timeit
wtc2 = np.where(
    df['Start Date'].isnull().values,
    (df['Original Record: Date Created'].values - df['Date Created']).dt.days / 7,
    (df['Date Created'].values - df['Start Date']).dt.days / 7
)

10 loops, best of 3: 47.5 ms per loop


Another approach is to do ndarray type casting, converting our series into numpy timedelta arrays. This way is faster, but more verbose and kinda more code for basically the same thing.

In [46]:
%%timeit
wtc3 = np.where(
    df['Start Date'].isnull().values,
    ((df['Original Record: Date Created'].values - df['Date Created'].values).astype('timedelta64[D]') / np.timedelta64(1, 'D')) / 7,
    ((df['Date Created'].values - df['Start Date'].values).astype('timedelta64[D]') / np.timedelta64(1, 'D')) / 7
)

10 loops, best of 3: 21.2 ms per loop


#### Needing values on other rows for the logic
This example comes from a task I had to recreate a function like this in Excel:
```excel
=IF(A2=A1, IF(L2-L1 < 5, 0, 1), 1))
```
Where the `A` column is for ids, and the `L` column is for dates.

In [48]:
def time_looper(df):
    """ Using a plain Python for loop"""
    output = []
    for i, row in enumerate(range(0, len(df))):
        if i > 0:
            
            # compare the current id to the row above
            if df.iloc[i]['Internal ID'] == df.iloc[i-1]['Internal ID']:
                
                # compare the current date to the row above
                if (df.iloc[i]['Date Created'] - df.iloc[i-1]['Original Record: Date Created']).days < 5:
                    output.append(0)
                else:
                    output.append(1)
            else:
                output.append(1)
        else:
            output.append(np.nan)
    return output

In [49]:
def time_looper2(df):
    """Using pandas dataframe `.iterrows()` method for iterating over rows"""
    output = []
    for i, row in df.iterrows():
        if i > 0:
            if df.iloc[i]['Internal ID'] == df.iloc[i-1]['Internal ID']:
                if (df.iloc[i]['Date Created'] - df.iloc[i-1]['Original Record: Date Created']).days < 5:
                    output.append(0)
                else:
                    output.append(1)
            else:
                output.append(1)
        else:
            output.append(np.nan)
    return output

In [50]:
df.sort_values(['Internal ID', 'Date Created'], inplace=True)

In [51]:
%%time
df['time_col_raw_for'] = time_looper(df)

Wall time: 5min 41s


In [52]:
%%time
df['time_col_iterrows'] = time_looper2(df)

Wall time: 4min 45s


Our approach to vectorizing this unfortunate situation was two-fold:
 1. Using the pandas `.shift()` function, we moved previous values down so they're on the same axis as what we're comparing them to 
 2. `np.select()` for the vectorized conditional logic check
 

In [53]:
%%timeit
previous_id = df['Internal ID'].shift(1).fillna(0).astype(int)
previous_date = df['Original Record: Date Created'].shift(1).fillna(pd.Timestamp('1900'))

conditions = [
    ((df['Internal ID'].values ==  previous_id) & 
     (df['Date Created'] - previous_date).astype('timedelta64[D]') < 5),
    df['Internal ID'].values ==  previous_id
]
choices = [0, 1]
df['time_col1'] = np.select(conditions, choices, default=1)

10 loops, best of 3: 39.2 ms per loop


In [54]:
(df['time_col1'] == df['time_col_iterrows']).all()

False

# Conclusions: 
1. Look for a method where you can invoke numpy.where, numpy.select or numpy.vectorize, and you can get dramatic speed gainst compared to .apply, or simple nested for loops in Python. 
2. Almost always, a method can be developed which can eliminate .apply
    